In [2]:
%load_ext autoreload
%autoreload
import surreal, config
import tensorflow as tf
import numpy as np
import keras
from keras import layers as KL

tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth=True
keras.backend.tensorflow_backend.set_session(tf.Session(config=tf_config))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [21]:
TENSOR_INFO = [
    {
        'name': 'image',
        'shape': (config.BATCH_SIZE, None, config.TAR_H, config.TAR_W, 3),
        'type': tf.uint8,
    },{
        'name': 'heatmap',
        'shape': (config.BATCH_SIZE, None, config.TAR_H, config.TAR_W, config.NUM_KP),
        'type': tf.uint8,
    },{
        'name': 'so_x',
        'shape': (config.BATCH_SIZE, None, config.TAR_H, config.TAR_W, config.NUM_KP),
        'type': tf.int32,
    },{
        'name': 'so_y',
        'shape': (config.BATCH_SIZE, None, config.TAR_H, config.TAR_W, config.NUM_KP),
        'type': tf.int32,
    },{
        'name': 'mo_x',
        'shape': (config.BATCH_SIZE, None, config.TAR_H, config.TAR_W, config.NUM_EDGE),
        'type': tf.int32,
    },{
        'name': 'mo_y',
        'shape': (config.BATCH_SIZE, None, config.TAR_H, config.TAR_W, config.NUM_EDGE),
        'type': tf.int32,
    }]

TOTAL_SIZE = sum(info['shape'][-1] for info in TENSOR_INFO)

input_tensors = tf.data.Dataset.from_generator(surreal.load, tf.int32) \
                               .batch(config.BATCH_SIZE) \
                               .prefetch(config.PREFETCH_SIZE) \
                               .make_one_shot_iterator() \
                               .get_next()

input_tensors.set_shape((config.BATCH_SIZE, None, config.TAR_H, config.TAR_W, TOTAL_SIZE))

input_layer = KL.Input(tensor=tf.cast(input_tensors, tf.float32))
print(input_layer)
input_layer = KL.TimeDistributed(KL.Reshape((320 * 320 * 113,), input_shape=(320, 320, 113)))(input_layer)

input_layer

Tensor("Cast_21:0", shape=(2, ?, 320, 320, 113), dtype=float32)


<tf.Tensor 'time_distributed_6/transpose_1:0' shape=(2, ?, 11571200) dtype=float32>

In [35]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions

class ResnetCell(KL.Layer):
    def __init__(self, units, state_size, **kwargs):
        self.units = units
        self.state_size = state_size
        super(ResnetCell, self).__init__(**kwargs) 

    def build(self, input_shape):
        self.model = ResNet50(include_top=False)
        self.model.outputs = [self.model.layers[-1].output]
        self.recurrent_kernel = self.add_weight(
            shape=(self.units, self.units),
            initializer='uniform',
            name='recurrent_kernel')
        self.built = True
        
        self.heatmap_conv = KL.Conv2D(config.NUM_KP, kernel_size=(1, 1), activation='relu')
        self.s_off_x_conv = KL.Conv2D(config.NUM_KP, kernel_size=(1, 1))
        self.s_off_y_conv = KL.Conv2D(config.NUM_KP, kernel_size=(1, 1))
        self.m_off_x_conv = KL.Conv2D(config.NUM_EDGE, kernel_size=(1, 1))
        self.m_off_y_conv = KL.Conv2D(config.NUM_EDGE, kernel_size=(1, 1))
        
    def call(self, inputs, states):
        reshaped = KL.Reshape((320, 320, 113), input_shape=(320 * 320 * 113,))(inputs)
        splited = tf.split(reshaped, [info['shape'][-1] for info in TENSOR_INFO], axis=-1)
        img, heatmap, so_x, so_y, mo_x, mo_y = splited
        
        tf.image.resize_bicubic(input_tensor, size=(config.TAR_H, config.TAR_W))
        model_output = self.model(img)
        print(model_output)
        return output, [output]
    

cell = ResnetCell(123, 234)
layer = keras.layers.RNN(cell, return_sequences=True)
y = layer(input_layer)


NameError: name 'input_tensor' is not defined

In [10]:

mm = keras.applications.mobilenetv2.MobileNetV2(input_shape=(224, 224, 3), weights=None, include_top=False)
mm.layers

In [11]:
mm.layers[-2].output

<tf.Tensor 'Conv_1_bn_2/cond/Merge:0' shape=(?, 7, 7, 1280) dtype=float32>

In [3]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions

resnet_model = ResNet50(include_top=False,
                        input_tensor=tensors['image'],
                        weights=None,
                        pooling=None,
                        )
res_out = resnet_model.layers[-2].output

In [4]:
def bilinear_resize(input_tensor): # resizes input tensor wrt. ref_tensor
    return tf.image.resize_bicubic(input_tensor, size=(config.TAR_H, config.TAR_W))

heatmap = KL.Conv2D(config.NUM_KP, kernel_size=(1, 1), activation='relu')(res_out)
heatmap = KL.Lambda(bilinear_resize)(heatmap)

s_off_x = KL.Conv2D(config.NUM_KP, kernel_size=(1, 1))(res_out)
s_off_x = KL.Lambda(bilinear_resize)(s_off_x)

s_off_y = KL.Conv2D(config.NUM_KP, kernel_size=(1, 1))(res_out)
s_off_y = KL.Lambda(bilinear_resize)(s_off_y)

m_off_x = KL.Conv2D(config.NUM_EDGE, kernel_size=(1, 1))(res_out)
m_off_x = KL.Lambda(bilinear_resize)(m_off_x)

m_off_y = KL.Conv2D(config.NUM_EDGE, kernel_size=(1, 1))(res_out)
m_off_y = KL.Lambda(bilinear_resize)(m_off_y)


In [5]:
config.NUM_RECURSIVE = 4

def adjust_recursive(args):
    s_off_x, s_off_y, m_off_x, m_off_y = args
    x = np.expand_dims(np.tile(np.arange(config.TAR_W), [1, config.TAR_H, 1]), axis=-1)
    y = np.expand_dims(np.tile(np.arange(config.TAR_H), [1, config.TAR_W, 1]), axis=-1).transpose()
    dx = m_off_x + x
    dy = m_off_y + y
    b_i = np.tile(np.arange(config.BATCH_SIZE), [config.NUM_EDGE, config.TAR_W, config.TAR_H, 1]).transpose()
    e_i = np.tile(config.EDGES[..., 0], [config.BATCH_SIZE, config.TAR_W, config.TAR_H, 1])
    
    for _ in range(config.NUM_RECURSIVE):
        dx_i = tf.cast(tf.round(dx), 'int64')
        dy_i = tf.cast(tf.round(dy), 'int64')
        dx_i = keras.backend.maximum(dx_i, 0)
        dx_i = keras.backend.minimum(dx_i, config.TAR_W - 1)
        dy_i = keras.backend.maximum(dy_i, 0)
        dy_i = keras.backend.minimum(dy_i, config.TAR_H - 1)
        ii = tf.stack([b_i, dx_i, dy_i, e_i], axis=-1)
        dx = dx + tf.gather_nd(s_off_x, ii)
        dy = dy + tf.gather_nd(s_off_y, ii)
    m_off_x = dx - x
    m_off_y = dy - x
    return keras.backend.stack([m_off_x, m_off_y], axis=0)

def split_x(res):
    return res[0, ...]
def split_y(res):
    return res[1, ...]

res = keras.layers.Lambda(adjust_recursive)([s_off_x, s_off_y, m_off_x, m_off_y])
m_off_x = keras.layers.Lambda(split_x)(res)
m_off_y = keras.layers.Lambda(split_y)(res)

In [6]:
config.EDGES[..., 0]

array([ 0,  0,  1,  1,  2,  3,  4,  5,  6,  7,  8,  5,  6, 11, 12, 13, 14,
        5, 11])

In [7]:
m_off_x._keras_history

(<keras.layers.core.Lambda at 0x12a62e6a0>, 0, 0)

In [8]:

losses = []

def heatmap_loss(args):
    true, pred, exclude_mask = args
    loss = keras.backend.binary_crossentropy(tf.cast(true, tf.float32), pred)
    loss = keras.backend.mean(loss, axis=-1, keepdims=True)
    exclude = tf.expand_dims(tf.cast(exclude_mask, tf.float32), axis=-1)
    loss = loss * exclude
    loss = keras.backend.sum(loss) / (keras.backend.sum(exclude) + keras.backend.epsilon())
    loss = loss * config.HEATMAP_LOSS_WEIGHT
    return loss

losses.append(keras.layers.Lambda(heatmap_loss)([tensors['heatmap'], heatmap, tensors['exclude_mask']]))

def short_offset_loss(args):
    true, pred, exclude_mask, true_disc = args
    loss = keras.backend.abs(tf.cast(true, tf.float32) - pred) / config.RADIUS
    exclude = tf.expand_dims(tf.cast(exclude_mask, tf.float32), axis=-1)
    exclude = exclude * tf.cast(true_disc, tf.float32)
    loss = loss * exclude
    loss = keras.backend.sum(loss) / (keras.backend.sum(exclude) + keras.backend.epsilon())
    loss = loss * config.SHORT_OFFSET_LOSS_WEIGHT
    return loss

losses.append(keras.layers.Lambda(short_offset_loss)([tensors['so_x'], s_off_x, tensors['exclude_mask'], tensors['heatmap']]))
losses.append(keras.layers.Lambda(short_offset_loss)([tensors['so_y'], s_off_y, tensors['exclude_mask'], tensors['heatmap']]))

def mid_offset_loss(args):
    true, pred, exclude_mask, true_disc = args
    loss = keras.backend.abs(tf.cast(true, tf.float32) - pred) / config.RADIUS
    exclude = tf.expand_dims(tf.cast(exclude_mask, tf.float32), axis=-1)
    exclude = exclude * tf.cast(tf.gather(true_disc, config.EDGES[..., 0], axis=-1), tf.float32)
    loss = loss * exclude
    loss = keras.backend.sum(loss) / (keras.backend.sum(exclude) + keras.backend.epsilon())
    loss = loss * config.MIDDLE_OFFSET_LOSS_WEIGHT
    return loss

losses.append(keras.layers.Lambda(mid_offset_loss)([tensors['mo_x'], m_off_x, tensors['exclude_mask'], tensors['heatmap']]))
losses.append(keras.layers.Lambda(mid_offset_loss)([tensors['mo_y'], m_off_y, tensors['exclude_mask'], tensors['heatmap']]))
print(input_tensors)
model = keras.models.Model(inputs = [tensors[t['name']] for t in TENSOR_INFO],
#                           outputs = [heatmap])
                           outputs = [heatmap, s_off_x, s_off_y, m_off_x, m_off_y, res_out])

(<tf.Tensor 'IteratorGetNext:0' shape=(2, 401, 401, 3) dtype=uint8>, <tf.Tensor 'IteratorGetNext:1' shape=(2, 401, 401) dtype=uint8>, <tf.Tensor 'IteratorGetNext:2' shape=(2, 401, 401, 17) dtype=uint8>, <tf.Tensor 'IteratorGetNext:3' shape=(2, 401, 401, 17) dtype=int32>, <tf.Tensor 'IteratorGetNext:4' shape=(2, 401, 401, 17) dtype=int32>, <tf.Tensor 'IteratorGetNext:5' shape=(2, 401, 401, 19) dtype=int32>, <tf.Tensor 'IteratorGetNext:6' shape=(2, 401, 401, 19) dtype=int32>)


In [11]:
if config.NUM_GPUS > 1:
    model = keras.utils.multi_gpu_model(model, config.NUM_GPUS)

ValueError: To call `multi_gpu_model` with `gpus=2`, we expect the following devices to be available: ['/cpu:0', '/gpu:0', '/gpu:1']. However this machine only has: ['/cpu:0']. Try reducing `gpus`.

In [9]:
config.NUM_EPOCHS = 500
print(losses)
for loss in losses:
    model.add_loss(loss)
SAVE_MODEL_FILE = 'model_output'
model.compile(optimizer=keras.optimizers.Adam(), target_tensors=None, loss=None)

def save_model(epoch, logs):
    model.save_weights(SAVE_MODEL_FILE)
callbacks = [keras.callbacks.LambdaCallback(on_epoch_end=save_model)]

model.fit(steps_per_epoch=100,
          epochs=config.NUM_EPOCHS,
          callbacks=callbacks)

[<tf.Tensor 'lambda_9/mul_1:0' shape=() dtype=float32>, <tf.Tensor 'lambda_10/mul_2:0' shape=() dtype=float32>, <tf.Tensor 'lambda_11/mul_2:0' shape=() dtype=float32>, <tf.Tensor 'lambda_12/mul_2:0' shape=() dtype=float32>, <tf.Tensor 'lambda_13/mul_2:0' shape=() dtype=float32>]


/Users/minsubsim/.virtualenvs/cv/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Output "lambda_1" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "lambda_1" during training.
  
/Users/minsubsim/.virtualenvs/cv/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Output "lambda_2" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "lambda_2" during training.
  
/Users/minsubsim/.virtualenvs/cv/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Output "lambda_3" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "lambda_3" during training.
  
/Users/minsubsim/.virtualenvs/cv/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Output "lambda_7" missing from loss dictionary. We assume this was done on purpose, and we will n

Epoch 1/500
loading annotations into memory...
Done (t=0.35s)
creating index...
index created!
100/100 [==============================] - 1174s 12s/step - loss: 4.5271
Epoch 2/500
100/100 [==============================] - 1119s 11s/step - loss: 3.9906
Epoch 3/500
100/100 [==============================] - 1106s 11s/step - loss: 3.9064
Epoch 4/500
100/100 [==============================] - 1028s 10s/step - loss: 3.4378
Epoch 5/500
100/100 [==============================] - 998s 10s/step - loss: 3.1104
Epoch 6/500
100/100 [==============================] - 991s 10s/step - loss: 2.8752
Epoch 7/500
100/100 [==============================] - 970s 10s/step - loss: 2.6492
Epoch 8/500
100/100 [==============================] - 965s 10s/step - loss: 2.4206
Epoch 9/500
 10/100 [==>...........................] - ETA: 20:37 - loss: 2.4940

KeyboardInterrupt: 

In [ ]:
outputs = model.predict(x=None,
                       steps=1,)

In [ ]:
outputs.shape

In [ ]:
from matplotlib import pyplot as plt

def overlay(img, over, alpha=0.5):
    out = img.copy()
    if img.max() > 1.:
        out = out / 255.
    out *= 1-alpha
    if len(over.shape)==2:
        out += alpha*over[:,:,np.newaxis]
    else:
        out += alpha*over    
    return out
Rshoulder_map = outputs[0,:,:,0]
print(Rshoulder_map)
plt.figure()
plt.imshow(overlay(img, Rshoulder_map, alpha=0.7))

In [ ]:
t1 = np.full((config.NUM_KP, 3), 2)
t2 = np.full(config.NUM_KP, 0)

In [ ]:
t1[np.arange(config.NUM_KP), t2[np.arange(config.NUM_KP)]:]